In [1]:
def send_to_telegram(message,chatID):
    apiToken = '6146608659:AAGtU4ny-4xBgUeLuZUS4yT8nNTdkfeW2GU'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        #print(response.text)
    except Exception as e:
        print(e)
    
    sleep(5)

In [2]:
def getUser():
    apiToken = '6146608659:AAGtU4ny-4xBgUeLuZUS4yT8nNTdkfeW2GU'
    apiURL = f'https://api.telegram.org/bot{apiToken}/getUpdates'
    
    response = requests.get(apiURL)
    content = response.content.decode("utf8")
    js = json.loads(content)
    return js['result'][0]['message']['from']['id']

In [3]:
def getMsg():
    apiToken = '6146608659:AAGtU4ny-4xBgUeLuZUS4yT8nNTdkfeW2GU'
    apiURL = f'https://api.telegram.org/bot{apiToken}/getUpdates'
    
    response = requests.get(apiURL)
    content = response.content.decode("utf8")
    js = json.loads(content)
    return js['result'][-1]['message']['text']

In [8]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from time import sleep
import json
import requests

firebase_admin.delete_app(firebase_admin.get_app())

# Fetch the service account key JSON file contents
cred = credentials.Certificate('user-data-53c43-firebase-adminsdk-cp6kr-1a23a8f5ef.json')
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://user-data-53c43-default-rtdb.asia-southeast1.firebasedatabase.app"
})

chatID = ""
userId = ""
bike_ls = []

while(1):
    chatID = getUser()
    if (chatID!=""):
        userId = getMsg()
        userProfile = db.reference(f'/User_Database/{userId}').get()
        if (userProfile):
            for i in list(userProfile.values())[1]:
                if (i != 'EMPTYELEMENT' and i!=None):
                    bike_ls.append(i)
            break
        else:
            send_to_telegram("Enter your identification code.",chatID)

while(1):
    bike_no = 1
    for i in bike_ls:
        alarmState = db.reference(f'/Bike_Database/{i}/alarmState').get()
        if (alarmState == "1"):
            send_to_telegram(f"Bike {bike_no} Alarm triggered!",chatID)
        requestState = db.reference(f'/Bike_Database/{i}/requestState').get()
        lockState = db.reference(f'/Bike_Database/{i}/lockState').get()
        if (requestState != "-1" and requestState==lockState):
            if (lockState == "1"):
                send_to_telegram(f"Bike {bike_no} Locked Successfully.",chatID)
            elif (lockState == "0"):
                send_to_telegram(f"Bike {bike_no} Unlocked Successfully.",chatID)
        bike_no+=1

KeyboardInterrupt: 